In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import time 
from tensorflow.keras.models import Sequential, load_model
from keras.layers import SimpleRNN, Dense
import tensorflow as tf
%load_ext tensorboard

In [2]:
data_path_and_labels = pd.read_csv("../mapped_data/xalan-2.4.csv")
data_path_and_labels['data_path'][0]

'dataset/promise/defect data/xalan-2.4/xalan-j_2_4_0/src/org/apache/xalan/Version.java.restrictedcontent.embed'

In [3]:
data_path_and_labels['data_path'] = '../' + data_path_and_labels['data_path']
data_path_and_labels['data_path'][0]

'../dataset/promise/defect data/xalan-2.4/xalan-j_2_4_0/src/org/apache/xalan/Version.java.restrictedcontent.embed'

In [4]:
data_path_and_labels['data_path'] = data_path_and_labels['data_path'].str.replace('.embed','')
data_path_and_labels['data_path'][0]

'../dataset/promise/defect data/xalan-2.4/xalan-j_2_4_0/src/org/apache/xalan/Version.java.restrictedcontent'

In [5]:
words = []
for index, row in data_path_and_labels.iterrows():
    with open(row['data_path']) as f:
        lines = f.read().splitlines()
    words.append(lines)

In [6]:
d = pd.read_csv("../mapped_data/xalan-2.4.csv")
d['data_path'][0]

'dataset/promise/defect data/xalan-2.4/xalan-j_2_4_0/src/org/apache/xalan/Version.java.restrictedcontent.embed'

In [7]:
d['data_path'] = '../' + d['data_path']
d['data_path'][0]

'../dataset/promise/defect data/xalan-2.4/xalan-j_2_4_0/src/org/apache/xalan/Version.java.restrictedcontent.embed'

In [8]:
d

,data_path,bugs
0,../dataset/promise/defect data/xalan-2.4/xalan...,0
1,../dataset/promise/defect data/xalan-2.4/xalan...,2
2,../dataset/promise/defect data/xalan-2.4/xalan...,0
3,../dataset/promise/defect data/xalan-2.4/xalan...,0
4,../dataset/promise/defect data/xalan-2.4/xalan...,0
...,...,...
671,../dataset/promise/defect data/xalan-2.4/xalan...,0
672,../dataset/promise/defect data/xalan-2.4/xalan...,0
673,../dataset/promise/defect data/xalan-2.4/xalan...,0
674,../dataset/promise/defect data/xalan-2.4/xalan...,0


In [9]:
train, test = train_test_split(d, test_size=0.2)

In [10]:
test_x = []
for index, row in test.iterrows():
    with open(row['data_path']) as f:
        lines = list(map(float, f.read().splitlines())) 
    test_x.append(lines)

In [11]:
train_x = []
for index, row in train.iterrows():
    with open(row['data_path']) as f:
        lines = list(map(float, f.read().splitlines())) 
    train_x.append(lines)

In [12]:
vocab = float(max(np.max(train_x), np.max(test_x)))
vocab

9459.0

In [13]:
train_y = train[' bugs'].to_numpy()
test_y = test[' bugs'].to_numpy()

In [14]:
train_y

array([0, 0, 0, 0, 2, 2, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 3, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 2, 0, 0, 0, 0,
       0, 2, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 2, 0, 0, 0,
       0, 0, 0, 4, 2, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0,
       0, 1, 0, 0, 0, 2, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 2, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 3, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 2, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0,

In [15]:
train_x = np.array(train_x)
test_x = np.array(test_x)

In [16]:
train_x.shape

(540, 7367)

In [17]:
# define the model
model = Sequential()
model.add(tf.keras.layers.Embedding(10671, 30, input_length=train_x.shape[1]))
model.add(tf.keras.layers.Flatten())
model.add(Dense(1, activation='relu'))
# compile the model
model.compile(loss = 'mse' , optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001) , metrics = ['mse', 'mae', tf.keras.metrics.RootMeanSquaredError(), tf.keras.metrics.MeanSquaredLogarithmicError()] )

In [18]:
# Fitting the model on training data.
import datetime
log_dir = '/home/bavanya/Desktop/tensorboard/' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
history = model.fit(train_x, train_y, epochs = 100, callbacks=[tensorboard_callback])

Epoch 1/100
17/17 [==============================] - 1s 27ms/step - loss: 0.3768 - mse: 0.3768 - mae: 0.2685 - root_mean_squared_error: 0.6138 - mean_squared_logarithmic_error: 0.1286
Epoch 2/100
17/17 [==============================] - 0s 26ms/step - loss: 0.3722 - mse: 0.3722 - mae: 0.2278 - root_mean_squared_error: 0.6101 - mean_squared_logarithmic_error: 0.1233
Epoch 3/100
17/17 [==============================] - 0s 25ms/step - loss: 0.3722 - mse: 0.3722 - mae: 0.2278 - root_mean_squared_error: 0.6101 - mean_squared_logarithmic_error: 0.1233
Epoch 4/100
17/17 [==============================] - 0s 24ms/step - loss: 0.3722 - mse: 0.3722 - mae: 0.2278 - root_mean_squared_error: 0.6101 - mean_squared_logarithmic_error: 0.1233
Epoch 5/100
17/17 [==============================] - 0s 24ms/step - loss: 0.3722 - mse: 0.3722 - mae: 0.2278 - root_mean_squared_error: 0.6101 - mean_squared_logarithmic_error: 0.1233
Epoch 6/100
17/17 [==============================] - 0s 25ms/step - loss: 0.3722

17/17 [==============================] - 1s 32ms/step - loss: 0.3722 - mse: 0.3722 - mae: 0.2278 - root_mean_squared_error: 0.6101 - mean_squared_logarithmic_error: 0.1233
Epoch 45/100
17/17 [==============================] - 1s 31ms/step - loss: 0.3722 - mse: 0.3722 - mae: 0.2278 - root_mean_squared_error: 0.6101 - mean_squared_logarithmic_error: 0.1233
Epoch 46/100
17/17 [==============================] - 0s 25ms/step - loss: 0.3722 - mse: 0.3722 - mae: 0.2278 - root_mean_squared_error: 0.6101 - mean_squared_logarithmic_error: 0.1233
Epoch 47/100
17/17 [==============================] - 0s 23ms/step - loss: 0.3722 - mse: 0.3722 - mae: 0.2278 - root_mean_squared_error: 0.6101 - mean_squared_logarithmic_error: 0.1233
Epoch 48/100
17/17 [==============================] - 0s 27ms/step - loss: 0.3722 - mse: 0.3722 - mae: 0.2278 - root_mean_squared_error: 0.6101 - mean_squared_logarithmic_error: 0.1233
Epoch 49/100
17/17 [==============================] - 0s 26ms/step - loss: 0.3722 - mse:

17/17 [==============================] - 0s 24ms/step - loss: 0.3722 - mse: 0.3722 - mae: 0.2278 - root_mean_squared_error: 0.6101 - mean_squared_logarithmic_error: 0.1233
Epoch 89/100
17/17 [==============================] - 0s 25ms/step - loss: 0.3722 - mse: 0.3722 - mae: 0.2278 - root_mean_squared_error: 0.6101 - mean_squared_logarithmic_error: 0.1233
Epoch 90/100
17/17 [==============================] - 0s 29ms/step - loss: 0.3722 - mse: 0.3722 - mae: 0.2278 - root_mean_squared_error: 0.6101 - mean_squared_logarithmic_error: 0.1233
Epoch 91/100
17/17 [==============================] - 1s 31ms/step - loss: 0.3722 - mse: 0.3722 - mae: 0.2278 - root_mean_squared_error: 0.6101 - mean_squared_logarithmic_error: 0.1233
Epoch 92/100
17/17 [==============================] - 0s 29ms/step - loss: 0.3722 - mse: 0.3722 - mae: 0.2278 - root_mean_squared_error: 0.6101 - mean_squared_logarithmic_error: 0.1233
Epoch 93/100
17/17 [==============================] - 0s 25ms/step - loss: 0.3722 - mse:

In [19]:
%tensorboard --logdir '/home/bavanya/Desktop/tensorboard/'

Reusing TensorBoard on port 6006 (pid 21467), started 0:41:07 ago. (Use '!kill 21467' to kill it.)

In [20]:
score = model.evaluate(test_x, test_y)
score

5/5 [==============================] - 0s 6ms/step - loss: 0.7426 - mse: 0.7426 - mae: 0.2426 - root_mean_squared_error: 0.8618 - mean_squared_logarithmic_error: 0.1393


[0.7426470518112183,
 0.7426470518112183,
 0.24264705181121826,
 0.8617697358131409,
 0.13933055102825165]

In [21]:
data_path_and_labels = pd.read_csv("../mapped_data/xalan-2.4.csv")
data_path_and_labels['data_path'][0]

'dataset/promise/defect data/xalan-2.4/xalan-j_2_4_0/src/org/apache/xalan/Version.java.restrictedcontent.embed'

In [22]:
data_path_and_labels['data_path'] = '../' + data_path_and_labels['data_path']
data_path_and_labels['data_path'][0]

'../dataset/promise/defect data/xalan-2.4/xalan-j_2_4_0/src/org/apache/xalan/Version.java.restrictedcontent.embed'

In [23]:
embeddings = []
for index, row in data_path_and_labels.iterrows():
    with open(row['data_path']) as f:
        lines = list(map(float, f.read().splitlines())) 
    embeddings.append(lines)

In [24]:
df = pd.DataFrame(embeddings)
df

,0,1,2,3,4,5,6,7,8,9,...,7357,7358,7359,7360,7361,7362,7363,7364,7365,7366
0,2456.0,5449.0,2250.0,8094.0,5261.0,5004.0,5094.0,5281.0,4878.0,4878.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,994.0,5463.0,5756.0,8094.0,517.0,5175.0,2250.0,8094.0,519.0,4397.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1254.0,4397.0,2562.0,9316.0,2569.0,1020.0,1020.0,5139.0,2270.0,8663.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2448.0,2455.0,2455.0,8051.0,8019.0,5441.0,5756.0,8094.0,8599.0,4142.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1990.0,2455.0,6742.0,2455.0,6287.0,3529.0,6589.0,79.0,79.0,5756.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
671,4808.0,1806.0,8094.0,3805.0,1806.0,3805.0,3611.0,3611.0,5756.0,6167.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
672,4808.0,1806.0,8094.0,3805.0,1806.0,3805.0,3611.0,3611.0,5756.0,3611.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
673,4808.0,1806.0,8094.0,3805.0,1806.0,3805.0,3611.0,3611.0,5756.0,6167.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
674,4808.0,1806.0,8094.0,3805.0,1806.0,3805.0,3611.0,3611.0,5756.0,6167.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
data = pd.concat([data_path_and_labels, df], axis = 1)
data = data.drop('data_path', 1)
data

,bugs,0,1,2,3,4,5,6,7,8,...,7357,7358,7359,7360,7361,7362,7363,7364,7365,7366
0,0,2456.0,5449.0,2250.0,8094.0,5261.0,5004.0,5094.0,5281.0,4878.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,994.0,5463.0,5756.0,8094.0,517.0,5175.0,2250.0,8094.0,519.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,1254.0,4397.0,2562.0,9316.0,2569.0,1020.0,1020.0,5139.0,2270.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,2448.0,2455.0,2455.0,8051.0,8019.0,5441.0,5756.0,8094.0,8599.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,1990.0,2455.0,6742.0,2455.0,6287.0,3529.0,6589.0,79.0,79.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
671,0,4808.0,1806.0,8094.0,3805.0,1806.0,3805.0,3611.0,3611.0,5756.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
672,0,4808.0,1806.0,8094.0,3805.0,1806.0,3805.0,3611.0,3611.0,5756.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
673,0,4808.0,1806.0,8094.0,3805.0,1806.0,3805.0,3611.0,3611.0,5756.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
674,0,4808.0,1806.0,8094.0,3805.0,1806.0,3805.0,3611.0,3611.0,5756.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
train, test = train_test_split(data, test_size=0.2)

In [27]:
train

,bugs,0,1,2,3,4,5,6,7,8,...,7357,7358,7359,7360,7361,7362,7363,7364,7365,7366
305,0,1591.0,5756.0,269.0,5756.0,2889.0,1385.0,1431.0,2848.0,1431.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
75,0,316.0,1319.0,6748.0,3529.0,6585.0,316.0,5693.0,5693.0,5610.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
611,1,2016.0,3529.0,344.0,1419.0,6566.0,1218.0,6346.0,1218.0,2273.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
649,0,2386.0,5756.0,24.0,1724.0,6271.0,2386.0,6271.0,1724.0,7911.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
552,1,2428.0,2428.0,6600.0,6483.0,8485.0,3806.0,5756.0,4311.0,1806.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,0,1921.0,3327.0,5509.0,2268.0,4223.0,1020.0,7888.0,4223.0,4280.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
621,0,2528.0,2528.0,7042.0,2250.0,7042.0,2012.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
531,0,2453.0,4397.0,2562.0,9316.0,2569.0,4053.0,3529.0,2558.0,9336.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
511,0,2240.0,5756.0,6317.0,2240.0,9264.0,5756.0,7184.0,2250.0,7167.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
test

,bugs,0,1,2,3,4,5,6,7,8,...,7357,7358,7359,7360,7361,7362,7363,7364,7365,7366
301,0,1813.0,2250.0,2892.0,1813.0,1813.0,5987.0,2250.0,2892.0,5987.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
547,0,2513.0,3529.0,6562.0,2513.0,7419.0,1905.0,7419.0,2513.0,3771.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
39,0,1019.0,2250.0,6781.0,2250.0,6883.0,5266.0,2250.0,8094.0,6781.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50,0,1024.0,1938.0,6310.0,8543.0,9150.0,2455.0,5756.0,7118.0,8599.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
229,2,2266.0,2250.0,3164.0,1938.0,2953.0,2250.0,3109.0,2266.0,3022.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,0,1260.0,8237.0,3357.0,1063.0,3371.0,5756.0,8044.0,3635.0,3371.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
618,0,2255.0,5756.0,1386.0,5756.0,6320.0,2250.0,6643.0,5756.0,6935.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
593,0,450.0,3529.0,461.0,3529.0,344.0,433.0,6445.0,433.0,5756.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
500,0,4397.0,2562.0,9316.0,2569.0,8094.0,2194.0,4587.0,9186.0,5985.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [29]:
Y_train = train[' bugs']
Y_test = test[' bugs']

In [30]:
train_y = Y_train.to_numpy()
test_y = Y_test.to_numpy()

In [31]:
train = train.drop(' bugs', axis=1)
test = test.drop(' bugs', axis=1)

In [32]:
X_train = train.values
X_test = test.values

In [33]:
train_x = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
test_x = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

In [34]:
# Designing and initializing the model.
model = Sequential()
model.add(SimpleRNN(100, input_shape = (1,20), dropout = 0.2, return_sequences=True))
model.add(SimpleRNN(80, dropout = 0.2, return_sequences=True))
model.add(SimpleRNN(60, dropout = 0.2, return_sequences=False))
model.add(Dense(1, activation = 'relu'))
model.compile(loss = 'mse' , optimizer = tf.keras.optimizers.Adam(learning_rate=0.00001) , metrics = ['mse', 'mae', tf.keras.metrics.RootMeanSquaredError(), tf.keras.metrics.MeanSquaredLogarithmicError()] )

In [35]:
# Fitting the model on training data.
history = model.fit(train_x, train_y, epochs = 100)

Epoch 1/100
17/17 [==============================] - 2s 7ms/step - loss: 0.5345 - mse: 0.5345 - mae: 0.4250 - root_mean_squared_error: 0.7311 - mean_squared_logarithmic_error: 0.1894
Epoch 2/100
17/17 [==============================] - 0s 7ms/step - loss: 0.5224 - mse: 0.5224 - mae: 0.4049 - root_mean_squared_error: 0.7228 - mean_squared_logarithmic_error: 0.1830
Epoch 3/100
17/17 [==============================] - 0s 7ms/step - loss: 0.4631 - mse: 0.4631 - mae: 0.3793 - root_mean_squared_error: 0.6805 - mean_squared_logarithmic_error: 0.1646
Epoch 4/100
17/17 [==============================] - 0s 7ms/step - loss: 0.4525 - mse: 0.4525 - mae: 0.3639 - root_mean_squared_error: 0.6727 - mean_squared_logarithmic_error: 0.1581
Epoch 5/100
17/17 [==============================] - 0s 7ms/step - loss: 0.4888 - mse: 0.4888 - mae: 0.3723 - root_mean_squared_error: 0.6991 - mean_squared_logarithmic_error: 0.1702
Epoch 6/100
17/17 [==============================] - 0s 7ms/step - loss: 0.4165 - mse

17/17 [==============================] - 0s 7ms/step - loss: 0.3624 - mse: 0.3624 - mae: 0.2517 - root_mean_squared_error: 0.6020 - mean_squared_logarithmic_error: 0.1161
Epoch 46/100
17/17 [==============================] - 0s 6ms/step - loss: 0.3793 - mse: 0.3793 - mae: 0.2591 - root_mean_squared_error: 0.6159 - mean_squared_logarithmic_error: 0.1228
Epoch 47/100
17/17 [==============================] - 0s 7ms/step - loss: 0.3665 - mse: 0.3665 - mae: 0.2532 - root_mean_squared_error: 0.6054 - mean_squared_logarithmic_error: 0.1166
Epoch 48/100
17/17 [==============================] - 0s 6ms/step - loss: 0.3796 - mse: 0.3796 - mae: 0.2617 - root_mean_squared_error: 0.6161 - mean_squared_logarithmic_error: 0.1222
Epoch 49/100
17/17 [==============================] - 0s 7ms/step - loss: 0.3873 - mse: 0.3873 - mae: 0.2652 - root_mean_squared_error: 0.6223 - mean_squared_logarithmic_error: 0.1247
Epoch 50/100
17/17 [==============================] - 0s 7ms/step - loss: 0.3540 - mse: 0.354

17/17 [==============================] - 0s 7ms/step - loss: 0.3545 - mse: 0.3545 - mae: 0.2390 - root_mean_squared_error: 0.5954 - mean_squared_logarithmic_error: 0.1127
Epoch 90/100
17/17 [==============================] - 0s 7ms/step - loss: 0.3421 - mse: 0.3421 - mae: 0.2408 - root_mean_squared_error: 0.5849 - mean_squared_logarithmic_error: 0.1092
Epoch 91/100
17/17 [==============================] - 0s 7ms/step - loss: 0.3598 - mse: 0.3598 - mae: 0.2470 - root_mean_squared_error: 0.5998 - mean_squared_logarithmic_error: 0.1127
Epoch 92/100
17/17 [==============================] - 0s 6ms/step - loss: 0.3731 - mse: 0.3731 - mae: 0.2576 - root_mean_squared_error: 0.6108 - mean_squared_logarithmic_error: 0.1195
Epoch 93/100
17/17 [==============================] - 0s 7ms/step - loss: 0.3163 - mse: 0.3163 - mae: 0.2273 - root_mean_squared_error: 0.5624 - mean_squared_logarithmic_error: 0.0992
Epoch 94/100
17/17 [==============================] - 0s 8ms/step - loss: 0.3220 - mse: 0.322

In [36]:
score = model.evaluate(test_x, test_y)
score

5/5 [==============================] - 0s 3ms/step - loss: 0.6339 - mse: 0.6339 - mae: 0.2512 - root_mean_squared_error: 0.7962 - mean_squared_logarithmic_error: 0.1297


[0.6339470744132996,
 0.6339470744132996,
 0.25124266743659973,
 0.7962079048156738,
 0.1297411024570465]